In this build, we are building a xgboost model. 

The imputation methods are:
- Categorical: fill in with "NA"
- Numeric: fill in with interoplate linear

In [3]:
import pandas as pd
import scipy as sp
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
import xgboost as xgb
import pprint as pp
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
import bnp_helper
import common_helper
import random
import os
import datetime
from sklearn.externals import joblib
from matplotlib.pylab import rcParams
from scipy.stats import randint as sp_randint

%matplotlib inline
rcParams['figure.figsize'] = 16, 4
pd.set_option('display.max_columns', 500)

In [5]:
################### Combine the training set and test set to fix the unmatched categories ############
train_df_raw = pd.read_csv("../data/train.csv")
test_df_raw = pd.read_csv("../data/test.csv")

df_raw_combined = pd.concat([train_df_raw, test_df_raw], axis = 0)

########### Clean and Impute (Combined) #############
df_combinded = bnp_helper.clean(df_raw_combined, drop_collinearity = True, inplace = True)
df_combinded = bnp_helper.impute_cate_with_na_numeric_with_outlier(df_combinded)

################ Convert text to number (Combined) ################
df_combinded = common_helper.dummify(df_combinded, bnp_helper.get_categorical_variables(df_combinded))
print df_combinded.shape
df_combinded.head()

(228714, 449)


,ID,target,v1,v10,v101,v102,v103,v106,v108,v109,v114,v117,v118,v119,v120,v122,v123,v124,v126,v127,v129,v13,v130,v131,v14,v16,v18,v19,v2,v21,v23,v27,v28,v35,v36,v37,v38,v4,v42,v43,v44,v45,v49,v50,v51,v53,v55,v57,v58,v59,v6,v60,v61,v62,v63,v65,v68,v69,v7,v70,v72,v73,v76,v77,v78,v80,v81,v82,v84,v85,v86,v87,v88,v9,v90,v93,v94,v95,v96,v98,v99,v107_B,v107_C,v107_D,v107_E,v107_F,v107_G,v107_NA,v110_B,v110_C,v112_B,v112_C,v112_D,v112_E,v112_F,v112_G,v112_H,v112_I,v112_J,v112_K,v112_L,v112_M,v112_N,v112_NA,v112_O,v112_P,v112_Q,v112_R,v112_S,v112_T,v112_U,v112_V,v113_AA,v113_AB,v113_AC,v113_AD,v113_AE,v113_AF,v113_AG,v113_AH,v113_AI,v113_AJ,v113_AK,v113_B,v113_C,v113_D,v113_E,v113_F,v113_G,v113_H,v113_I,v113_J,v113_K,v113_L,v113_M,v113_N,v113_NA,v113_O,v113_P,v113_Q,v113_R,v113_S,v113_T,v113_U,v113_V,v113_W,v113_X,v113_Y,v113_Z,v125_AA,v125_AB,v125_AC,v125_AD,v125_AE,v125_AF,v125_AG,v125_AH,v125_AI,v125_AJ,v125_AK,v125_AL,v125_AM,v125_AN,v125_AO,v125_AP,v125_AQ,v125_AR,v125_AS,v125_AT,v125_AU,v125_AV,v125_AW,v125_AX,v125_AY,v125_AZ,v125_B,v125_BA,v125_BB,v125_BC,v125_BD,v125_BE,v125_BF,v125_BG,v125_BH,v125_BI,v125_BJ,v125_BK,v125_BL,v125_BM,v125_BN,v125_BO,v125_BP,v125_BQ,v125_BR,v125_BS,v125_BT,v125_BU,v125_BV,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_NA,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z,v24_B,v24_C,v24_D,v24_E,v3_B,v3_C,v3_NA,v30_B,v30_C,v30_D,v30_E,v30_F,v30_G,v30_NA,v31_B,v31_C,v31_NA,v47_B,v47_C,v47_D,v47_E,v47_F,v47_G,v47_H,v47_I,v47_J,v52_B,v52_C,v52_D,v52_E,v52_F,v52_G,v52_H,v52_I,v52_J,v52_K,v52_L,v52_NA,v56_AA,v56_AB,v56_AC,v56_AD,v56_AE,v56_AF,v56_AG,v56_AH,v56_AI,v56_AJ,v56_AK,v56_AL,v56_AM,v56_AN,v56_AO,v56_AP,v56_AQ,v56_AR,v56_AS,v56_AT,v56_AU,v56_AV,v56_AW,v56_AX,v56_AY,v56_AZ,v56_B,v56_BA,v56_BB,v56_BC,v56_BD,v56_BE,v56_BF,v56_BG,v56_BH,v56_BI,v56_BJ,v56_BK,v56_BL,v56_BM,v56_BN,v56_BO,v56_BP,v56_BQ,v56_BR,v56_BS,v56_BT,v56_BU,v56_BV,v56_BW,v56_BX,v56_BY,v56_BZ,v56_C,v56_CA,v56_CB,v56_CC,v56_CD,v56_CE,v56_CF,v56_CG,v56_CH,v56_CI,v56_CJ,v56_CK,v56_CL,v56_CM,v56_CN,v56_CO,v56_CP,v56_CQ,v56_CR,v56_CS,v56_CT,v56_CU,v56_CV,v56_CW,v56_CX,v56_CY,v56_CZ,v56_D,v56_DA,v56_DB,v56_DC,v56_DD,v56_DE,v56_DF,v56_DG,v56_DH,v56_DI,v56_DJ,v56_DK,v56_DL,v56_DM,v56_DN,v56_DO,v56_DP,v56_DQ,v56_DR,v56_DS,v56_DT,v56_DU,v56_DV,v56_DW,v56_DX,v56_DY,v56_DZ,v56_E,v56_F,v56_G,v56_H,v56_I,v56_J,v56_K,v56_L,v56_M,v56_N,v56_NA,v56_O,v56_P,v56_Q,v56_R,v56_S,v56_T,v56_U,v56_V,v56_W,v56_X,v56_Y,v56_Z,v66_B,v66_C,v71_B,v71_C,v71_D,v71_E,v71_F,v71_G,v71_H,v71_I,v71_J,v71_K,v71_L,v74_B,v74_C,v75_B,v75_C,v75_D,v79_B,v79_C,v79_D,v79_E,v79_F,v79_G,v79_H,v79_I,v79_J,v79_K,v79_L,v79_M,v79_N,v79_O,v79_P,v79_Q,v79_R,v91_B,v91_C,v91_D,v91_E,v91_F,v91_G,v91_NA
0,3,1,1.335739,0.503281,8.389237,2.757375,4.374296,12.579184,2.382692,3.930922,15.634907,6.592012,5.909091,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,1.804126,3.113719,0,2.866830,0.636365,2.857144,11.636387,8.571429,0.106720,0.148883,8.727474,7.730923,-1.716131e-08,3.393503,0.590122,8.375452,11.326592,0.454546,0,3.921026,12.707581,2.015505,10.498338,9.848672,8.086643,0.899420,7.277792,16.747968,1.299638,3.971118,0.529802,10.890984,2.599278,1.588448,15.858152,1,0.153461,18.303925,15.231789,17.142857,3.176895,11.784549,1,1.614988,2.230940,7.292418,8.571429,3.000000,7.528326,8.861647,1.299638,1.707317,0.866426,9.551836,3.321300,9.999999,0.905342,5.814018,3.517720,0.462019,7.436824,8.877414,1.191337,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [75]:
############## Get samples for training ##################
train_df = df_combinded[-df_raw_combined['target'].isnull()]
test_df = df_combinded[df_raw_combined['target'].isnull()]

train_df_sample = train_df.sample(10000, random_state = 0)
#train_df_sample = train_df
target_train = train_df_sample['target']
X_train = train_df_sample.drop(['ID', 'target'], axis = 1)


In [70]:
from sklearn import svm

svm_model = svm.SVC(probability = True, random_state = 27)

start = datetime.datetime.now()
svm_model.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

predprob_train = svm_model.predict_proba(X_train)
metrics.log_loss(target_train, predprob_train)


model training time: 0:00:00.414683


0.50966006453207713

In [66]:
print 2.0 ** np.arange(-5, 16, 3)
print 2.0 ** np.arange(-15, 4, 3)
print 2.0 ** np.arange(-5, 4, 3)
print 2.0 ** np.arange(-15, -4, 3)


[  3.12500000e-02   2.50000000e-01   2.00000000e+00   1.60000000e+01
   1.28000000e+02   1.02400000e+03   8.19200000e+03]
[  3.05175781e-05   2.44140625e-04   1.95312500e-03   1.56250000e-02
   1.25000000e-01   1.00000000e+00   8.00000000e+00]
[ 0.03125  0.25     2.     ]
[  3.05175781e-05   2.44140625e-04   1.95312500e-03   1.56250000e-02]


In [49]:
svm_model = svm.SVC(probability = True, random_state = 27)

# coarse grid search
para_grid = {
    'gamma': 2.0 ** np.arange(-15, 4, 2), 
    'C': 2.0 ** np.arange(-5, 16, 2)
}


n_iter_search = 20

random_search = RandomizedSearchCV(svm_model, param_distributions=para_grid, scoring = "log_loss",
                                   n_iter=n_iter_search, n_jobs = 4)

start = datetime.datetime.now()
random_search.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

pp.pprint(random_search.grid_scores_)

# best combinatin
print 'Best combination: ', random_search.best_params_

# best score
print 'Best Score' , random_search.best_score_


model training time: 1:38:34.439198
[mean: -0.53204, std: 0.00121, params: {'C': 2.0, 'gamma': 0.125},
 mean: -0.54502, std: 0.00073, params: {'C': 8192.0, 'gamma': 0.5},
 mean: -0.55019, std: 0.00049, params: {'C': 32.0, 'gamma': 2.0},
 mean: -0.53327, std: 0.00099, params: {'C': 512.0, 'gamma': 0.125},
 mean: -0.54508, std: 0.00035, params: {'C': 32.0, 'gamma': 0.5},
 mean: -0.53161, std: 0.00083, params: {'C': 0.125, 'gamma': 0.03125},
 mean: -0.53155, std: 0.00083, params: {'C': 0.03125, 'gamma': 0.03125},
 mean: -0.53284, std: 0.00217, params: {'C': 0.5, 'gamma': 0.125},
 mean: -0.55104, std: 0.00022, params: {'C': 512.0, 'gamma': 8.0},
 mean: -0.55067, std: 0.00010, params: {'C': 2.0, 'gamma': 8.0}]
Best combination:  {'C': 0.03125, 'gamma': 0.03125}
Best Score -0.531551257787


In [76]:
################# Coarse Tune using Coarse Grid Search ##############
param_test = {
    'gamma': 2.0 ** np.arange(-15, -4, 3), 
    'C': 2.0 ** np.arange(-5, 16, 3)
}

gsearch = GridSearchCV(estimator = svm.SVC(probability = True, random_state = 27), \
    param_grid = param_test, scoring='log_loss',n_jobs=4,iid=False, cv=5)

start = datetime.datetime.now()
gsearch.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

pp.pprint(gsearch.grid_scores_)

# best combinatin
print 'Best combination: ', gsearch.best_params_

# best score
print 'Best Score' , gsearch.best_score_


model training time: 1:32:25.645038
[mean: -0.55129, std: 0.00245, params: {'C': 0.03125, 'gamma': 3.0517578125e-05},
 mean: -0.54828, std: 0.00665, params: {'C': 0.03125, 'gamma': 0.000244140625},
 mean: -0.54288, std: 0.00297, params: {'C': 0.03125, 'gamma': 0.001953125},
 mean: -0.53989, std: 0.00396, params: {'C': 0.03125, 'gamma': 0.015625},
 mean: -0.55731, std: 0.01042, params: {'C': 0.25, 'gamma': 3.0517578125e-05},
 mean: -0.54734, std: 0.00322, params: {'C': 0.25, 'gamma': 0.000244140625},
 mean: -0.54321, std: 0.00246, params: {'C': 0.25, 'gamma': 0.001953125},
 mean: -0.53980, std: 0.00390, params: {'C': 0.25, 'gamma': 0.015625},
 mean: -0.55451, std: 0.00186, params: {'C': 2.0, 'gamma': 3.0517578125e-05},
 mean: -0.54838, std: 0.00254, params: {'C': 2.0, 'gamma': 0.000244140625},
 mean: -0.54313, std: 0.00250, params: {'C': 2.0, 'gamma': 0.001953125},
 mean: -0.53926, std: 0.00407, params: {'C': 2.0, 'gamma': 0.015625},
 mean: -0.55336, std: 0.00110, params: {'C': 16.0, 'g

In [67]:
################# Coarse Tune using Grid Search ##############
param_test = {
    'gamma': 2.0 ** np.arange(-5, 4, 3), 
    'C': 2.0 ** np.arange(-5, 16, 3)
}

gsearch = GridSearchCV(estimator = svm.SVC(probability = True, random_state = 27), \
    param_grid = param_test, scoring='log_loss',n_jobs=4,iid=False, cv=5)

start = datetime.datetime.now()
gsearch.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

pp.pprint(gsearch.grid_scores_)

# best combinatin
print 'Best combination: ', gsearch.best_params_

# best score
print 'Best Score' , gsearch.best_score_


model training time: 1:35:34.493784
[mean: -0.53613, std: 0.00574, params: {'C': 0.03125, 'gamma': 0.03125},
 mean: -0.53824, std: 0.00382, params: {'C': 0.03125, 'gamma': 0.25},
 mean: -0.55384, std: 0.00008, params: {'C': 0.03125, 'gamma': 2.0},
 mean: -0.53626, std: 0.00519, params: {'C': 0.25, 'gamma': 0.03125},
 mean: -0.53813, std: 0.00398, params: {'C': 0.25, 'gamma': 0.25},
 mean: -0.55375, std: 0.00009, params: {'C': 0.25, 'gamma': 2.0},
 mean: -0.53620, std: 0.00510, params: {'C': 2.0, 'gamma': 0.03125},
 mean: -0.53811, std: 0.00317, params: {'C': 2.0, 'gamma': 0.25},
 mean: -0.55393, std: 0.00045, params: {'C': 2.0, 'gamma': 2.0},
 mean: -0.53869, std: 0.00452, params: {'C': 16.0, 'gamma': 0.03125},
 mean: -0.53808, std: 0.00311, params: {'C': 16.0, 'gamma': 0.25},
 mean: -0.55376, std: 0.00009, params: {'C': 16.0, 'gamma': 2.0},
 mean: -0.54036, std: 0.00416, params: {'C': 128.0, 'gamma': 0.03125},
 mean: -0.53807, std: 0.00319, params: {'C': 128.0, 'gamma': 0.25},
 mean: 

In [83]:
#X_train_scaled = preprocessing.scale(X_train)


In [94]:
################# Fine Tune using Grid Search ##############
param_test = {
    'gamma': [2**-13, 2**-12, 2**-11], 
    'C': [2**9, 2**10, 2**11]
}

gsearch = GridSearchCV(estimator = svm.SVC(probability = True, random_state = 27), \
    param_grid = param_test, scoring='log_loss',n_jobs=4,iid=False, cv=5)

start = datetime.datetime.now()
gsearch.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

pp.pprint(gsearch.grid_scores_)

# best combinatin
print 'Best combination: ', gsearch.best_params_

# best score
print 'Best Score' , gsearch.best_score_

model training time: 0:35:37.793269
[mean: -0.53363, std: 0.00413, params: {'C': 512, 'gamma': 0.0001220703125},
 mean: -0.53290, std: 0.00426, params: {'C': 512, 'gamma': 0.000244140625},
 mean: -0.53519, std: 0.00406, params: {'C': 512, 'gamma': 0.00048828125},
 mean: -0.53228, std: 0.00526, params: {'C': 1024, 'gamma': 0.0001220703125},
 mean: -0.53345, std: 0.00415, params: {'C': 1024, 'gamma': 0.000244140625},
 mean: -0.53633, std: 0.00410, params: {'C': 1024, 'gamma': 0.00048828125},
 mean: -0.53270, std: 0.00554, params: {'C': 2048, 'gamma': 0.0001220703125},
 mean: -0.53522, std: 0.00382, params: {'C': 2048, 'gamma': 0.000244140625},
 mean: -0.53748, std: 0.00375, params: {'C': 2048, 'gamma': 0.00048828125}]
Best combination:  {'C': 1024, 'gamma': 0.0001220703125}
Best Score -0.532275443495


In [95]:
################# Fine Tune using Grid Search ##############
param_test = {
    'gamma': [2**-13-0.00001,2**-13,2**-13+0.00001 ], 
    'C': [2**10-100, 2**10, 2**10+100]
}

gsearch = GridSearchCV(estimator = svm.SVC(probability = True, random_state = 27), \
    param_grid = param_test, scoring='log_loss',n_jobs=4,iid=False, cv=5)

start = datetime.datetime.now()
gsearch.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

pp.pprint(gsearch.grid_scores_)

# best combinatin
print 'Best combination: ', gsearch.best_params_

# best score
print 'Best Score' , gsearch.best_score_

model training time: 0:31:45.821371
[mean: -0.53241, std: 0.00498, params: {'C': 924, 'gamma': 0.0001120703125},
 mean: -0.53230, std: 0.00509, params: {'C': 924, 'gamma': 0.0001220703125},
 mean: -0.53231, std: 0.00513, params: {'C': 924, 'gamma': 0.0001320703125},
 mean: -0.53227, std: 0.00519, params: {'C': 1024, 'gamma': 0.0001120703125},
 mean: -0.53228, std: 0.00526, params: {'C': 1024, 'gamma': 0.0001220703125},
 mean: -0.53226, std: 0.00517, params: {'C': 1024, 'gamma': 0.0001320703125},
 mean: -0.53225, std: 0.00534, params: {'C': 1124, 'gamma': 0.0001120703125},
 mean: -0.53221, std: 0.00535, params: {'C': 1124, 'gamma': 0.0001220703125},
 mean: -0.53220, std: 0.00519, params: {'C': 1124, 'gamma': 0.0001320703125}]
Best combination:  {'C': 1124, 'gamma': 0.0001320703125}
Best Score -0.53219898794


In [ ]:
################# Fine Tune using Grid Search ##############
param_test = {
    'gamma': [0.00012707, 0.00013207], 
    'C': [1124, 1224, 1324]
}

gsearch = GridSearchCV(estimator = svm.SVC(probability = True, random_state = 27), \
    param_grid = param_test, scoring='log_loss',n_jobs=4,iid=False, cv=5)

start = datetime.datetime.now()
gsearch.fit(X_train, target_train)
end = datetime.datetime.now()

print "model training time: {}".format(end - start)

pp.pprint(gsearch.grid_scores_)

# best combinatin
print 'Best combination: ', gsearch.best_params_

# best score
print 'Best Score' , gsearch.best_score_

In [ ]:
# best estimator
best_model = para_search.best_estimator_
best_model.fit(X_train, target_train)

test_df_sample = test_df.sample(1000, random_state = 0)
#test_df_sample = test_df

X_test = test_df_sample.drop(['ID', 'target'], axis = 1)

predprob_one_test = logit_best.predict_proba(X_test)

In [ ]:
# best estimator
best_model = para_search.best_estimator_
best_model.fit(X_train, target_train)

test_df_sample = test_df.sample(1000, random_state = 0)
#test_df_sample = test_df

X_test = test_df_sample.drop(['ID', 'target'], axis = 1)

predprob_one_test = logit_best.predict_proba(X_test)


In [ ]:
joblib.dump(xgb_model, 'svm_model.pkl')

In [ ]:
# Save results
ids = test_df_sample['ID']
predprob_one_test = [p[1] for p in predprob_test]
bnp_helper.save_result(ids, predprob_one_test, "xgboost_model_2_result.csv")

print('Done.')
